In [1]:
# Import the pandas library
import pandas as pd

# Load office address data from a CSV file
offices = 'D:/Python Projects/Consolidating Employee Data/office_addresses.csv'
offices_data = pd.read_csv(offices)
print(offices_data.head())

# Load employee information from an Excel file
emp_info = 'D:/Python Projects/Consolidating Employee Data/employee_information.xlsx'
emp_info_data = pd.ExcelFile(emp_info)
print(emp_info_data.sheet_names)

# Parse the first and second sheets from the Excel file
emp_address = emp_info_data.parse(0)
emp_emergy_contacts = emp_info_data.parse(1)

          office office_country    office_city   office_street  \
0  Leuven Office             BE         Leuven  Martelarenlaan   
1     ESB Office             US  New York City    Fifth Avenue   
2  WeWork Office             GB         London      Old Street   

   office_street_number  
0                    38  
1                   350  
2                   207  
['employee_addresses', 'emergency_contacts']


In [3]:
# Import the JSON library
import json

# Load employee roles data from a JSON file
emp_roles_path = 'D:/Python Projects/Consolidating Employee Data/employee_roles.json'
with open(emp_roles_path, 'r') as json_file:
    emp_roles_data = json.load(json_file)

In [4]:
# Define the columns to read from the Excel file
addresses_cols = ['employee_id', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number']

# Read specified columns from the Excel file into a DataFrame
addresses = pd.read_excel(emp_info, usecols=addresses_cols)
print(addresses.head())

# Define the header for the emergency contact DataFrame
emergy_contact_header = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']

# Read specified columns from the Excel file into a DataFrame
emergency_contacts = pd.read_excel(emp_info, sheet_name=1, header=None, names=emergy_contact_header)
print(emergency_contacts.head())

  employee_id employee_country employee_city      employee_street  \
0      A2R5H9               BE        Leuven          Grote Markt   
1      H8K0L6               GB        London         Baker Street   
2      G4R7V0               US      New-York         Perry Street   
3      M1Z7U9               FR         Paris  Rue de l'Université   

   employee_street_number  
0                       9  
1                     221  
2                      66  
3                       7  
  employee_id last_name first_name emergency_contact emergency_contact_number  \
0      A2R5H9    Hunman        Jax        Opie Hurst          +32-456-5556-84   
1      H8K0L6      Siff       Tara   Wendy de Matteo         +44-020-5554-333   
2      G4R7V0     Sagal      Gemma      John Newmark           +1-202-555-194   
3      M1Z7U9    Coates        Tig       Venus Noone          +1-202-555-0130   

  relationship  
0      Brother  
1       Sister  
2      Husband  
3         Wife  


In [5]:
# Read employee roles from the JSON file into a DataFrame
emp_roles = pd.read_json(emp_roles_path, orient="index")
print(emp_roles)

                     title monthly_salary               team
A2R5H9                 CEO          $4500         Leadership
H8K0L6                 CFO          $4500         Leadership
G4R7V0  Business Developer          $3000              Sales
M1Z7U9      Office Manager          $2000  People Operations


In [6]:
# Perform a left join between the "addresses" and "offices_data" DataFrames
employees = addresses.merge(offices_data, left_on='employee_country', right_on='office_country', how='left')
print(employees.head())

# Merge the "employees" DataFrame with the "emp_roles" DataFrame using the employee_id as the key
employees = employees.merge(emp_roles, left_on='employee_id', right_index=True, how='inner')
print(employees.head())

# Merge the "employees" DataFrame with the "emergency_contacts" DataFrame using the employee_id as the key
employees = employees.merge(emergency_contacts, on='employee_id', how='inner')
print(employees.head())



  employee_id employee_country employee_city      employee_street  \
0      A2R5H9               BE        Leuven          Grote Markt   
1      H8K0L6               GB        London         Baker Street   
2      G4R7V0               US      New-York         Perry Street   
3      M1Z7U9               FR         Paris  Rue de l'Université   

   employee_street_number         office office_country    office_city  \
0                       9  Leuven Office             BE         Leuven   
1                     221  WeWork Office             GB         London   
2                      66     ESB Office             US  New York City   
3                       7            NaN            NaN            NaN   

    office_street  office_street_number  
0  Martelarenlaan                  38.0  
1      Old Street                 207.0  
2    Fifth Avenue                 350.0  
3             NaN                   NaN  
  employee_id employee_country employee_city      employee_street  \
0   

In [7]:
# Find null values in the "employees" DataFrame
null = employees.isna()
print(null.sum())

# Replace null values in columns starting with "office" with "Remote"
for column in employees.columns:
    if column.startswith('office'):
        employees[column].fillna('Remote', inplace=True)

# Check if there are still null values
null = employees.isna()
print(null.sum())




employee_id                 0
employee_country            0
employee_city               0
employee_street             0
employee_street_number      0
office                      1
office_country              1
office_city                 1
office_street               1
office_street_number        1
title                       0
monthly_salary              0
team                        0
last_name                   0
first_name                  0
emergency_contact           0
emergency_contact_number    0
relationship                0
dtype: int64
employee_id                 0
employee_country            0
employee_city               0
employee_street             0
employee_street_number      0
office                      0
office_country              0
office_city                 0
office_street               0
office_street_number        0
title                       0
monthly_salary              0
team                        0
last_name                   0
first_name                 

In [8]:
# Drop the "last_name" and "first_name" columns from the DataFrame
employees.drop(columns=['last_name', 'first_name'], inplace=True)
print(employees)

# Check the current column names in the DataFrame
column_names = employees.columns.tolist()
print(column_names)



  employee_id employee_country employee_city      employee_street  \
0      A2R5H9               BE        Leuven          Grote Markt   
1      H8K0L6               GB        London         Baker Street   
2      G4R7V0               US      New-York         Perry Street   
3      M1Z7U9               FR         Paris  Rue de l'Université   

   employee_street_number         office office_country    office_city  \
0                       9  Leuven Office             BE         Leuven   
1                     221  WeWork Office             GB         London   
2                      66     ESB Office             US  New York City   
3                       7         Remote         Remote         Remote   

    office_street office_street_number               title monthly_salary  \
0  Martelarenlaan                 38.0                 CEO          $4500   
1      Old Street                207.0                 CFO          $4500   
2    Fifth Avenue                350.0  Business Dev

In [9]:
# Rename columns to match the final column names
employees.rename(columns={
    'employee_id': 'id',
    'employee_country': 'country',
    'employee_city': 'city',
    'employee_street': 'street',
    'employee_street_number': 'street_number'
}, inplace=True)

# Check if column names have changed
column_names = employees.columns.tolist()
print(column_names)



['id', 'country', 'city', 'street', 'street_number', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number', 'title', 'monthly_salary', 'team', 'emergency_contact', 'emergency_contact_number', 'relationship']


In [10]:
# Define a list of final columns to subset the data
final_columns = ['id', 'country', 'city', 'street', 'street_number', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number', 'title', 'monthly_salary', 'team', 'emergency_contact', 'emergency_contact_number', 'relationship']

# Subset the data for the final columns
employees_final = employees[final_columns]

# Set the "id" column as the index
employees_final.set_index('id', inplace=True)

print(employees_final)

       country      city               street  street_number         office  \
id                                                                            
A2R5H9      BE    Leuven          Grote Markt              9  Leuven Office   
H8K0L6      GB    London         Baker Street            221  WeWork Office   
G4R7V0      US  New-York         Perry Street             66     ESB Office   
M1Z7U9      FR     Paris  Rue de l'Université              7         Remote   

       office_country    office_city   office_street office_street_number  \
id                                                                          
A2R5H9             BE         Leuven  Martelarenlaan                 38.0   
H8K0L6             GB         London      Old Street                207.0   
G4R7V0             US  New York City    Fifth Avenue                350.0   
M1Z7U9         Remote         Remote          Remote               Remote   

                     title monthly_salary               team  